In [16]:
import pyWinhook as pyHook
import pythoncom
import pandas as pd
import numpy as np
import csv
import PySimpleGUI as sg

In [27]:
# make a dataframe
df = pd.DataFrame(columns=['Key', 'Ascii', 'Time'])
userFilePath = 'myInputCsv.csv'

In [30]:

class KeyLogger:
    def __init__(self):
        self.enterPressed = False
        self.eventList = []
        self.isCaps = False
        self.userName = getUserName()
        self.idOfWord = 1
        #self.message = ""

    def keyDownEvent(self, event):
        if event.KeyID == 20 and self.isCaps == False:
            self.isCaps = True
        elif event.KeyID == 20 and self.isCaps == True:
            self.isCaps = False     
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        self.storeEvent("Down", event) 
        return True
    

    #Currently not used
    def mainLoop(self):
        while not self.enterPressed:
            pythoncom.PumpWaitingMessages()

            
    def keyUpEvent(self, event):
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        print(chr(event.Ascii),end='')
        self.storeEvent("Up", event)
        return True


    def storeEvent(self, activity, event):
        keystrokeTime = int(event.Time)

        # Makes sure the enter key up and down have the same idOfWord
        if event.Ascii == 13 and activity == "Up":
            self.eventList.append ((self.userName, self.idOfWord - 1, event.Ascii, event.Key, activity, int(keystrokeTime)))
        else:
            self.eventList.append ((self.userName, self.idOfWord, event.Ascii, event.Key, activity, int(keystrokeTime)))

       
        
        
        # Chosen to use Escape key (ESC) due to input using a similar method
        # Enter Key - KeyCode: 13 Ascii: 13 ScanCode: 28 - ESC = 27 @ Ascii
        # Activity condition had to be added because the userRecordData function was being called twice because of the keyUpEvent and keyDownEvent
        # if event.Ascii == 13 and activity == "Down": 
        #     self.enterPressed = True
        #     print("Enter Pressed")
        #     userRecordData(self.eventList)

def userRecordData(eventList):
    print("\nOutput:")
    print(eventList)
    
    # Delete the first element of the list because the first element is always a keyUpEvent for the Enter key
    eventList.pop(0)
    
    header = ["user", "idOfWord", "ascii", "key", "keyEvent", "Time"]  # Header list
    with open(userFilePath,'a',newline='\n') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(eventList)
    f.close()


#Not needed currently
def getKeyStroke():
    
    keyLogger = KeyLogger()
    hookManager = pyHook.HookManager()
    hookManager.KeyDown = keyLogger.keyDownEvent
    hookManager.KeyUp = keyLogger.keyUpEvent
    hookManager.HookKeyboard()

    keyLogger.mainLoop()
    # Unhooks the keyboard, no more data recorded, returns to menu
    hookManager.UnhookKeyboard()


def getUserName():
        userName = input("Enter your name: ")
        return userName


In [31]:
sg.theme('DarkAmber')   # Add a touch of color

#TODO: 1) Make it so when the username is entered the Up event of the enter key is not recorded

# Create the keylogger and hook it to the keyboard
keyLogger = KeyLogger()
hookManager = pyHook.HookManager()
hookManager.KeyDown = keyLogger.keyDownEvent
hookManager.KeyUp = keyLogger.keyUpEvent
hookManager.HookKeyboard()

# All the stuff inside your window.
layout = [  [sg.Text('Collecting KeyStroke Data for User: ' + keyLogger.userName)],
            [sg.Text('Enter password (.tie5Roanl)'), sg.InputText( key='Input1')],
            [sg.Button('Ok'), sg.Button('Cancel'), sg.Push(), sg.Button('Make csv')] ]

# Create the Window
window = sg.Window('Window Title', layout, finalize=True)

# Bind the Enter key to the input field -> Used to clear the input field when enter is pressed
# For this to work, when creating the window, finalize must be set to True
window['Input1'].bind('<Return>', '_Enter')

# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
    elif event == 'Input1' + '_Enter':
        window['Input1'].update('')
        keyLogger.idOfWord += 1
    elif event == 'Make csv': # This implementation prevents the same data from being recorded twice
        userRecordData(keyLogger.eventList)
        break
    
    #keyLogger.mainLoop()
    pythoncom.PumpWaitingMessages() # This is needed for the keylogger to work, and works without the mainLoop function

    #print('You entered ', values[0])
    print(values)

# Unhooks the keyboard, no more data recorded, returns to menu
hookManager.UnhookKeyboard()
window.close()

.TIE4r OANL{'Input1': '.tie4Roanl'}
.TIE5 ROANL{'Input1': '.tie5Roanl'}
.TIE5 ROANL{'Input1': '.tie5Roanl'}
.TIE5 ROANL{'Input1': '.tie5Roanl'}

Output:
[('vuko', 0, 13, 'Return', 'Up', 957971500), ('vuko', 1, 46, 'Oem_Period', 'Down', 957974593), ('vuko', 1, 46, 'Oem_Period', 'Up', 957974671), ('vuko', 1, 84, 'T', 'Down', 957974859), ('vuko', 1, 73, 'I', 'Down', 957974937), ('vuko', 1, 84, 'T', 'Up', 957974953), ('vuko', 1, 73, 'I', 'Up', 957975000), ('vuko', 1, 69, 'E', 'Down', 957975062), ('vuko', 1, 69, 'E', 'Up', 957975140), ('vuko', 1, 52, '4', 'Down', 957975468), ('vuko', 1, 52, '4', 'Up', 957975500), ('vuko', 1, 0, 'Lshift', 'Down', 957976421), ('vuko', 1, 114, 'R', 'Down', 957976562), ('vuko', 1, 114, 'R', 'Up', 957976640), ('vuko', 1, 0, 'Lshift', 'Up', 957976640), ('vuko', 1, 79, 'O', 'Down', 957976781), ('vuko', 1, 79, 'O', 'Up', 957976859), ('vuko', 1, 65, 'A', 'Down', 957976906), ('vuko', 1, 65, 'A', 'Up', 957976984), ('vuko', 1, 78, 'N', 'Down', 957977000), ('vuko', 1, 7

TASKS
1. Sučelje za prikupit inpute
    a. Osim svojstva pojedine akcije treba spremit i # repeticije riječi -> ne uzimat input za shift
    b. Spremit podatke u neki csv
2. Iz prikupljenih podataka izračunat H, DD, UD,...
3. Iz promjenjenih podataka napravit konačni  csv
    a. Gleda se ukupni input za jednu riječ/lozinku
3. Podijelit konačni csv na brojke i odvojenu u drugi csv koja osoba je to tipkala
4. Podijelit na test i train
5. Istrenirat mrežu
6. Sučelje za test
    a. Upišeš svoj username
    b. Nakon upisa šifre, izbacuje da li si ti ta osoba ili ne